In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [32]:
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')
data_test = pd.read_csv("..\\Data\\cle\\Rand_test.csv", sep=',')

In [33]:
X_test = data_test.iloc[:,1:]
Y_test = data_test.iloc[:,0]

X2_test = X_test
Y2_test = Y_test

In [34]:
cut = 0

y1_test = (Y_test <= cut) * 1
y2_test = (Y_test > cut) * (-1) * (-Y_test)
Y2_test = np.stack((y1_test, y2_test), axis=-1)

In [35]:
from fit9_nn import mae_error

In [36]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
       
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = torch.tanh(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [37]:
PATH = '..\\Data\\checkpoints\\mytraining2.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [38]:
with torch.no_grad():
    net.eval()  
    
    test_inputs, test_labels = torch.FloatTensor(X2_test.values), torch.FloatTensor(Y2_test)
    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)      
    
    test_outputs = net.forward(test_inputs)   
    
    test_outputs2, test_labels2 = test_outputs.numpy(), test_labels.numpy()

In [39]:
y = test_labels2[:,1]
p = test_outputs2[:,0]
f2 = test_outputs2[:,1]
yhat = (1-p)*f2

In [42]:
print(np.mean(np.abs(yhat - y)))

282.1859


In [43]:
np.min(yhat)

18.989925

In [44]:
np.max(yhat)

69.943405

In [45]:
np.mean(yhat)

56.040283

In [46]:
y1 = y[y==0]
yh1 = yhat[y==0]
y2 = y[y>0]
yh2 = yhat[y>0]

In [47]:
print(np.mean(np.abs(yh1 - y1)))
print(np.mean(np.abs(yh2 - y2)))

45.2887
336.9863
